In [1]:
import os
import pandas as pd
import sagemaker_pyspark
#import matplotlib.pyplot as plt
#%matplotlib inline

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField,StringType, FloatType
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import create_map, struct
from sagemaker_pyspark import classpath_jars
#from PIL import Image

In [3]:
os.environ["PYTHONIOENCODING"] = "utf-8"

In [4]:
LANDED_PATH = 'F:\\Sistema\\Downloads\\bigDataProjefct\\landed\\'
MODELED_PATH = "F:\\Sistema\\Downloads\\bigDataProjefct\\modeled\\"

In [5]:
classpath = ":".join(sagemaker_pyspark.classpath_jars())

builder = SparkSession.builder.appName("Acidentes de Transito")
builder.config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
builder.config("spark.speculation", "false")
builder.config("spark.sql.parquet.compression.codec", "gzip")
builder.config("spark.debug.maxToStringFields", "100")
builder.config("spark.driver.extraClassPath", classpath)
builder.config("spark.driver.memory", "1g")
builder.config("spark.driver.cores", "1")
builder.config("spark.executor-memory", "20g")
builder.config("spark.executor.cores", "4")

builder.master("local[*]")

spark = builder.getOrCreate()

In [6]:
def merge_vitimas_info(df):
    if "vitimasfatais" in df and df['tipo']=='VÍTIMA FATAL':
        df['vitimas'] = df['vitimasfatais']
    return df

In [7]:
def drop_vitimas_fatais(df):
    if "vitimasfatais" in df:
        df = df.drop("vitimasfatais")
    return df

In [8]:
def set_dafault_value(df):
    for coluna in ['auto', 'moto', 'ciclom', 'ciclista', 'pedestre', 'onibus', 'caminhao', 'viatura', 'outros', 'vitimas']:
        if coluna in df:
            df[coluna] = pd.to_numeric(df[coluna], errors='coerce').fillna(0.0, inplace=False)
            
    if 'data' in df:
        df['data'] = pd.to_datetime(df['data'], errors='coerce').dt.strftime('%m/%d/%Y')
        
    if 'hora' in df:
        df['hora'] = pd.to_datetime(df['hora'], errors='coerce').dt.strftime("%H:%M")
        
    df['situacao'] = df['situacao'].replace('Err:512', 'SEM_DADO', regex=True)
    df['situacao'] = df['situacao'].replace('', 'SEM_DADO', regex=True)
    
    df = df.fillna('SEM_DADO', inplace=False)
    return df

In [9]:
all_dataset = [f'{LANDED_PATH}{file}' for file in os.listdir(LANDED_PATH) if '.csv' in file]
all_dataset

['F:\\Sistema\\Downloads\\bigDataProjefct\\landed\\acidentes-transito-2015.csv',
 'F:\\Sistema\\Downloads\\bigDataProjefct\\landed\\acidentes-transito-2016.csv',
 'F:\\Sistema\\Downloads\\bigDataProjefct\\landed\\acidentes2017.csv',
 'F:\\Sistema\\Downloads\\bigDataProjefct\\landed\\acidentes2018.csv']

In [72]:
"""
dataset_list = spark.sparkContext.parallelize(all_dataset)
.map(lambda file: pd.read_csv(file, sep=';'))
.map(lambda df: df[df['situacao'] != 'CANCELADA'])
.map(lambda df: df.apply(lambda row: row.strip() if isinstance(row, str) else row, axis=1))
.map(lambda df: df.drop(["endereco", "numero", "complemento", "descricao"], axis=1))
.map(lambda df: df.apply(lambda row: merge_vitimas_info(row), axis=1))
.map(lambda df: df.apply(lambda row: drop_vitimas_fatais(row), axis=1))
.map(lambda df: set_dafault_value(df))
.reduce(lambda df_1, df_2: df_1.append(df_2, ignore_index=True))
"""

'\ndataset_list = spark.sparkContext.parallelize(all_dataset)\n.map(lambda file: pd.read_csv(file, sep=\';\'))\n.map(lambda df: df[df[\'situacao\'] != \'CANCELADA\'])\n.map(lambda df: df.apply(lambda row: row.strip() if isinstance(row, str) else row, axis=1))\n.map(lambda df: df.drop(["endereco", "numero", "complemento", "descricao"], axis=1))\n.map(lambda df: df.apply(lambda row: merge_vitimas_info(row), axis=1))\n.map(lambda df: df.apply(lambda row: drop_vitimas_fatais(row), axis=1))\n.map(lambda df: set_dafault_value(df))\n.reduce(lambda df_1, df_2: df_1.append(df_2))\n'

In [10]:
dataset_list = spark.sparkContext.parallelize(all_dataset).map(lambda file: pd.read_csv(file, sep=';')).map(lambda df: df[df['situacao'] != 'CANCELADA']).map(lambda df: df.apply(lambda row: row.strip() if isinstance(row, str) else row, axis=1)).map(lambda df: df.drop(["endereco", "numero", "complemento", "descricao"], axis=1)).map(lambda df: df.apply(lambda row: merge_vitimas_info(row), axis=1)).map(lambda df: df.apply(lambda row: drop_vitimas_fatais(row), axis=1)).map(lambda df: set_dafault_value(df)).reduce(lambda df_1, df_2: df_1.append(df_2, ignore_index=True))

In [16]:
#dataset_list.to_csv(f'{MODELED_PATH}final_dataset.csv', sep=';', encoding='utf-8')
dataset_list.to_parquet(f'{MODELED_PATH}final_dataset.parquet', engine='fastparquet', compression='gzip')

In [29]:
"""
df = pd.read_csv(all_dataset[1], sep=';')
df = df[df['situacao'] != 'CANCELADA']
df = df.drop(["endereco", "numero", "complemento", "descricao"], axis=1)
df = df.apply(lambda row: merge_vitimas_info(row), axis=1)
df = df.apply(lambda row: drop_vitimas_fatais(row), axis=1)
df = df.apply(lambda row: row.strip() if isinstance(row, str) else row, axis=1)
#df = set_dafault_value(df)
#df.to_csv(f'{LANDED}aaaaaa.csv', sep=';', encoding='utf-8')
df = df[df['bairro'] == 'BOA VIAGEM']
df = df[df['situacao'] == 'PENDENTE']
df = df[df['tipo'] == 'FISCALIZAÇÃO']
df = df[df['data'] == '2016-04-02']
#df = df[df['data'] == '02/04/2016']
#df.loc[5]
df
"""

''